In [19]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm

In [85]:
import random
random.seed(1024)

In [23]:
df = pd.read_csv("Citadel-Datathon-2021/mode1data_pcaed.csv")
#df.head()
df_cols = list(df.columns)
'''
['edu',
 'employ',
 'LogGDPPerCapita',
 'cbrt_income',
 'pop',
 'tax',
 'Africa',
 'Americas',
 'Eastern Mediterranean',
 'Europe',
 'South-East Asia',
 'Western Pacific',
 'pca_1',
 'pca_2',
 'pca_3',
 'Year',
 'Location',
 'Value']
'''

"\n['edu',\n 'employ',\n 'LogGDPPerCapita',\n 'cbrt_income',\n 'pop',\n 'tax',\n 'Africa',\n 'Americas',\n 'Eastern Mediterranean',\n 'Europe',\n 'South-East Asia',\n 'Western Pacific',\n 'pca_1',\n 'pca_2',\n 'pca_3',\n 'Year',\n 'Location',\n 'Value']\n"

In [82]:
# preprocess, including time info

# world average for a year-ok
# world std for a year-ok
# location progression rate 2018-2017 => del
# progression pct = (current - base) / base => features_2013 == 0
# world progression rate 2018-2017-ok => del
# location 2018 / world 2018 (std) progression..?



# IGNORE ABOVE




# location annual progression - world annual (mean) progression from 2013 to 2018


df_ww_annual_avg = df.groupby(["Year"]).mean()
df_ww_annual_std = df.groupby(["Year"]).std()

def getProgress(dff,cols,years):
    d = dict()
    d["Year"] = list(years)
    for col in cols:
        res = [0] # init 2013
        L = list(dff[col])
        for i in range(1,len(L)):
            if L[i-1] == 0: 
                res.append(0)
            else:
                res.append((L[i] - L[i-1]) / L[i-1]) 
        d[col+"_prog"] = res
    return pd.DataFrame(d)

def vsWorld(countries,df_ww,dfs_country):
    L = []
    for i in range(len(countries)):
        d = dict()
        d["Location"] = np.repeat(countries[i],len(range(2013,2019)))
        d["Year"] = range(2013,2019)
        df_country = dfs_country[i]
        for X in numeric_cols:
            d[X+"_relprog"] = (df_country[X+"_prog"] - df_ww[X+"_prog"])
        L.append(pd.DataFrame(d))
    return pd.concat(L)
                   
    

numeric_cols = df_cols[:6]+df_cols[12:15]
df_ww_annual_progress = getProgress(df_ww_annual_avg,numeric_cols,df_ww_annual_avg.index)
df_count = df.groupby(["Location","Year"]).sum()
countries = df["Location"].unique()
dfs_country = [getProgress(df_count.loc[(c,)],numeric_cols,range(2013,2019)) for c in countries]
df_all = df.merge(vsWorld(countries, df_ww_annual_progress,dfs_country),how="left") 

df_all.head()


,edu,employ,LogGDPPerCapita,cbrt_income,pop,tax,Africa,Americas,Eastern Mediterranean,Europe,...,Value,edu_relprog,employ_relprog,LogGDPPerCapita_relprog,cbrt_income_relprog,pop_relprog,tax_relprog,pca_1_relprog,pca_2_relprog,pca_3_relprog
0,0.508801,0.479838,0.569910,0.242478,0.000809,0.460127,1.0,0.0,0.0,0.0,...,10.7,-0.018742,-0.021196,0.007683,0.004951,0.001064,-0.013653,0.179556,1.245221,-0.789458
1,0.508801,0.479838,0.569910,0.369193,0.033583,0.460127,0.0,1.0,0.0,0.0,...,10.9,-0.018742,-0.021196,0.007683,-0.020572,-0.009223,-0.013653,0.179556,1.245221,-0.789458
2,0.447146,0.492851,0.569910,0.296962,0.002100,0.765212,0.0,1.0,0.0,0.0,...,11.0,-0.005533,-0.000035,0.134459,0.002280,-0.004411,-0.001831,-0.820444,0.245221,-1.789458
3,0.026791,0.507733,0.145309,0.064277,0.011106,0.460127,1.0,0.0,0.0,0.0,...,11.8,-0.125006,0.036938,-0.001840,0.112234,0.021495,-0.013653,0.321079,-1.793884,-0.552895
4,0.508801,0.479838,0.341118,0.148248,0.036894,0.428837,1.0,0.0,0.0,0.0,...,11.8,-0.018742,-0.021196,0.030424,0.041602,0.014107,-0.057757,0.278263,1.001552,-0.983835


In [320]:
print(df_all.columns)

X_m2 = df_all.drop(['Value', 'Year','Location','edu','employ','LogGDPPerCapita','cbrt_income','pop',
                   'tax','pca_1','pca_2','pca_3'], axis=1)
#X_m2 = df_all.drop(['Location'], axis=1)
#X_m2 = X_m2.drop(X_m2[X_m2['Year'] == 2013].index)
#X_m2 = X_m2.drop(['Year'], axis=1)
#y_m2 = X_m2["Value"]
y_m2 = df_all["Value"]
#X_m2 = X_m2.drop(['Value'], axis=1)
X_m2

Index(['edu', 'employ', 'LogGDPPerCapita', 'cbrt_income', 'pop', 'tax',
       'Africa', 'Americas', 'Eastern Mediterranean', 'Europe',
       'South-East Asia', 'Western Pacific', 'pca_1', 'pca_2', 'pca_3', 'Year',
       'Location', 'Value', 'edu_relprog', 'employ_relprog',
       'LogGDPPerCapita_relprog', 'cbrt_income_relprog', 'pop_relprog',
       'tax_relprog', 'pca_1_relprog', 'pca_2_relprog', 'pca_3_relprog'],
      dtype='object')


,Africa,Americas,Eastern Mediterranean,Europe,South-East Asia,Western Pacific,edu_relprog,employ_relprog,LogGDPPerCapita_relprog,cbrt_income_relprog,pop_relprog,tax_relprog,pca_1_relprog,pca_2_relprog,pca_3_relprog
0,1.0,0.0,0.0,0.0,0.0,0.0,-0.018742,-0.021196,0.007683,0.004951,0.001064,-0.013653,0.179556,1.245221,-0.789458
1,0.0,1.0,0.0,0.0,0.0,0.0,-0.018742,-0.021196,0.007683,-0.020572,-0.009223,-0.013653,0.179556,1.245221,-0.789458
2,0.0,1.0,0.0,0.0,0.0,0.0,-0.005533,-0.000035,0.134459,0.002280,-0.004411,-0.001831,-0.820444,0.245221,-1.789458
3,1.0,0.0,0.0,0.0,0.0,0.0,-0.125006,0.036938,-0.001840,0.112234,0.021495,-0.013653,0.321079,-1.793884,-0.552895
4,1.0,0.0,0.0,0.0,0.0,0.0,-0.018742,-0.021196,0.030424,0.041602,0.014107,-0.057757,0.278263,1.001552,-0.983835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
890,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
891,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
892,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [312]:
np.random.get_state()[1][3]

3570887495

In [322]:
# lasso
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X_m2_train, X_m2_test,y_m2_train, y_m2_test = train_test_split(X_m2, y_m2, test_size =0.2, random_state = 123445) # with 2013, random_state = 42, alhpa = 0.001
m2 = linear_model.Lasso(alpha=0.01,max_iter=100000,tol=1e-1)

In [323]:
m2.fit(X_m2_train, y_m2_train)
R2_train = m2.score(X_m2_train, y_m2_train)
R2_test = m2.score(X_m2_test, y_m2_test)
n_train,p = X_m2_train.shape
n_test,_ = X_m2_test.shape
print("Training r-square:", m2.score(X_m2_train, y_m2_train))
print("Adjusted Training r-square:", 1-(1-R2_train)*(n_train-1)/(n_train-p-1))
print("Testing r-square:", m2.score(X_m2_test, y_m2_test))
print("Adjusted Testing r-square:", 1-(1-R2_test)*(n_test-1)/(n_test-p-1))

Training r-square: 0.4166743986057204
Adjusted Training r-square: 0.4041566818376029
Testing r-square: 0.45001507325239465
Adjusted Testing r-square: 0.3994029634289954


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22332.423084696915, tolerance: 7531.401071328672
  positive)
